# Importando Bibliotecas

In [ ]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.svm import SVC
from scipy.stats import shapiro
from scipy.stats import f_oneway
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from statsmodels.stats.multicomp import MultiComparison
from sklearn.model_selection import cross_val_score, KFold

# Avaliação dos Algoritimos
- Naïve Bayes: 93.80
- Árvore de Decisão: 98.20
- Regras: 97.40
- Regressão Logística: 94.60
- SVM: 98.80
- Redes Neurais: 99.60

## Turning dos Parâmetros com GridSearch

### Preparação dos Dados

In [ ]:
with open('credit.pkl', 'rb') as f:
    x_credit_treinamento, y_credit_treinamento, x_credit_teste, y_credit_teste = pickle.load(f)

In [ ]:
x_credit_treinamento.shape, y_credit_treinamento.shape, x_credit_teste.shape, y_credit_teste.shape

In [ ]:
x_credit = np.concatenate((x_credit_treinamento, x_credit_teste), axis = 0)

In [ ]:
x_credit.shape

In [ ]:
y_credit = np.concatenate((y_credit_treinamento,y_credit_teste), axis = 0)

In [ ]:
y_credit

### Árvore de Decisão

In [ ]:
DecisionTreeClassifier()

In [ ]:
parametros = {'criterion' : ['gini', 'entropy'], # métodos para medir a impureza de um nó
              'splitter': ['best', 'random'], #  escolher a divisão em cada nó
              'min_samples_split': [2, 5, 10], # número mínimo de amostras necessárias para dividir um nó
              'min_samples_leaf': [1, 5, 10] } # número mínimo de amostras necessário para ser considerado como uma folha (nó terminal) da árvore

In [ ]:
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=parametros)
grid_search.fit(x_credit, y_credit)
melhores_parametros = grid_search.best_params_
melhores_resultados = grid_search.best_score_
print(melhores_parametros)
print(melhores_resultados)

### Random Forest 

In [ ]:
parametros = {'criterion' : ['gini', 'entropy'], # métodos para medir a impureza de um nó
              'n_estimators': [10, 40, 100, 500], # número de árvores na floresta
              'min_samples_split': [2, 5, 10], # número mínimo de amostras necessárias para dividir um nó
              'min_samples_leaf': [1, 5, 10] } # número mínimo de amostras necessário para ser considerado como uma folha (nó terminal) da árvore

In [ ]:
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=parametros)
grid_search.fit(x_credit, y_credit)
melhores_parametros = grid_search.best_params_
melhores_resultados = grid_search.best_score_
print(melhores_parametros)
print(melhores_resultados)

### Knn

In [ ]:
parametros = {'n_neighbors' : [3, 5, 10, 20], #  número de vizinhos mais próximos considerados ao fazer uma previsão 
              'p': [1,2]} # distância a ser utilizada para calcular a proximidade entre pontos. Quando p=1 Manhattan, p=2 Euclidiana

In [ ]:
grid_search = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=parametros)
grid_search.fit(x_credit, y_credit)
melhores_parametros = grid_search.best_params_
melhores_resultados = grid_search.best_score_
print(melhores_parametros)
print(melhores_resultados)

### Regressão Logística

In [ ]:
parametros = {'tol' : [0.0001, 0.00001, 0.000001], #  tolerância para o critério de parada
              'C' : [1.0, 1.5, 2.0], # força da regularização no modelo, menores de C indicam uma regularização mais forte, o que pode ajudar a evitar overfitting, enquanto valores maiores indicam uma regularização mais fraca.
              'solver' : ['lbfgs', 'sag', 'saga']} # especifica o algoritmo a ser usado no problema de otimização

In [ ]:
grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=parametros)
grid_search.fit(x_credit, y_credit)
melhores_parametros = grid_search.best_params_
melhores_resultados = grid_search.best_score_
print(melhores_parametros)
print(melhores_resultados)

### SVM

In [ ]:
parametros = {'tol' : [0.001, 0.0001, 0.00001], #  tolerância para o critério de parada
              'C' : [1.0, 1.5, 2.0], #  controla a força da regularização no modelo SVM
              'kernel' : ['rbf', 'linear', 'poly', 'sigmoid']} # especifica o tipo de função de kernel a ser usado no modelo SVM, o kernel determina o mapeamento das características do espaço original para um espaço de características de maior dimensão

In [ ]:
grid_search = GridSearchCV(estimator=SVC(), param_grid=parametros)
grid_search.fit(x_credit, y_credit)
melhores_parametros = grid_search.best_params_
melhores_resultados = grid_search.best_score_
print(melhores_parametros)
print(melhores_resultados)

### Redes Neurais

In [ ]:
parametros = {'activation' :  ['relu', 'logistic', 'tahn'], # define a função de ativação a ser usada nas camadas ocultas da rede neural
              'solver' : ['adam', 'sgd'], # especifica o algoritmo de otimização a ser usado durante o treinamento da rede neural
              'batch_size' : [10, 56]} # define o número de amostras de dados que serão usadas em cada iteração durante o treinamento

In [ ]:
grid_search = GridSearchCV(estimator=MLPClassifier(), param_grid=parametros)
grid_search.fit(x_credit, y_credit)
melhores_parametros = grid_search.best_params_
melhores_resultados = grid_search.best_score_
print(melhores_parametros)
print(melhores_resultados)

### Validação Cruzada

from sklearn.model_selection import cross_val_predict, KFold

In [ ]:
resultados_knn = []
resultados_svm = []
resultados_arvore = []
resultados_logistica = []
resultados_rede_neural = []
resultados_random_forest = []

for i in range(30): # é utilizado 30 testes geralmente
    kfold = KFold(n_splits=10, shuffle=True, random_state=i)
    
    arvore = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=1, min_samples_split=5, splitter='best')
    scores = cross_val_score(arvore, x_credit, y_credit, cv = kfold)
    #print(scores) 
    #print(scores.mean())
    resultados_arvore.append(scores.mean())
    
    random_forest = RandomForestClassifier(criterion='entropy', min_samples_leaf=1, min_samples_split=5, n_estimators=10)
    score = cross_val_score(random_forest, x_credit, y_credit, cv=kfold)
    resultados_random_forest.append(score.mean())
    
    knn = KNeighborsClassifier()
    scores = cross_val_score(knn, x_credit, y_credit, cv = kfold)
    resultados_knn.append(scores.mean())
    
    logistica = LogisticRegression(C=1.0, solver='lbfgs', tol=0.0001)
    scores = cross_val_score(logistica, x_credit, y_credit, cv=kfold)
    resultados_logistica.append(scores.mean())
    
    svm = SVC(kernel='rbf', C=2.0)
    scores = cross_val_score(svm, x_credit, y_credit, cv=kfold)
    resultados_svm.append(scores.mean())
    
    rede_neural = MLPClassifier(activation='relu', batch_size=56, solver='adam')
    scores = cross_val_score(rede_neural, x_credit, y_credit, cv=kfold)
    resultados_rede_neural.append(scores.mean())
    

In [ ]:
resultados_knn, resultados_svm, resultados_arvore, resultados_logistica, resultados_rede_neural, resultados_random_forest 

In [ ]:
resultados = pd.DataFrame({'Arvore': resultados_arvore,
                          'Random Forest' : resultados_random_forest,
                          'KNN' : resultados_knn,
                          'Logistica' : resultados_logistica,
                          'SVM' : resultados_svm,
                          'Rede Neural' : resultados_rede_neural})
resultados

In [ ]:
resultados.describe()

In [ ]:
(resultados.std() / resultados.mean()) * 100

### Teste de Normalidade nos Resultados

In [ ]:
alpha = 0.05 # confiança do teste é de 95 default

from scipy.stats import shapiro

In [ ]:
shapiro(resultados_arvore), shapiro(resultados_random_forest), shapiro(resultados_knn), shapiro(resultados_logistica), shapiro(resultados_svm), shapiro(resultados_rede_neural)

In [ ]:
sns.displot(resultados_arvore, kind='kde' )

In [ ]:
sns.displot(resultados_random_forest, kind='kde' )

In [ ]:
sns.displot(resultados_knn, kind='kde' )

In [ ]:
sns.displot(resultados_logistica, kind='kde' )

In [ ]:
sns.displot(resultados_rede_neural, kind='kde') 

### Teste de Hipótese com ANOVA e Tukey

from scipy.stats import f_oneway

In [ ]:
_, p = f_oneway(resultados_arvore, resultados_random_forest, resultados_logistica, resultados_svm, resultados_knn, resultados_rede_neural)
p

In [ ]:
alpha = 0.05
if p <= alpha:
    print('Hipótese nula rejeitada. Os dados são diferentes')
else:
    print('Hipótese alternativa rejeitada. Os resultados são iguais')

In [ ]:
resultados_algoritimos = {'accurancy' : np.concatenate([resultados_arvore, resultados_random_forest, resultados_logistica, resultados_knn, resultados_svm, resultados_rede_neural]),
                          'algoritimos' : ['arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore', 'arvore',
                          'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 'random_forest', 
                          'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica', 'logistica',
                          'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn','knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn', 'knn',
                          'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv', 'smv',
                          'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural', 'rede_neural']}

In [ ]:
df_resultados = pd.DataFrame(resultados_algoritimos)

In [ ]:
df_resultados

from statsmodels.stats.multicomp import MultiComparison

In [ ]:
compara_algoritimos =  MultiComparison(df_resultados['accurancy'], df_resultados['algoritimos'])

In [ ]:
teste_estatistico = compara_algoritimos.tukeyhsd()
print(teste_estatistico) # se reject é true quer dizer que os algoritimos são diferentes

In [ ]:
resultados.mean()

In [ ]:
teste_estatistico.plot_simultaneous()

## Salvando um Classificador já treinado

In [ ]:
with open('credit.pkl', 'rb') as f:
    x_credit_treinamento, y_credit_treinamento, x_credit_teste, y_credit_teste = pickle.load(f)

In [ ]:
x_credit = np.concatenate((x_credit_treinamento, x_credit_teste), axis=0)
y_credit = np.concatenate((y_credit_treinamento, y_credit_teste), axis=0)


In [ ]:
x_credit.shape, y_credit.shape

from sklearn.neural_network import MLPClassifier

In [ ]:
classificador_rede_neural = MLPClassifier(activation='relu', batch_size=56, solver='adam')
classificador_rede_neural.fit(x_credit, y_credit)

In [ ]:
classificador_arvore = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=1,  min_samples_split=5, splitter='best')
classificador_arvore.fit(x_credit, y_credit)

In [ ]:
classificador_svm = SVC(C=2.0, kernel='rbf')
classificador_svm.fit(x_credit, y_credit)

In [ ]:
pickle.dump(classificador_rede_neural, open('rede_neural_finalizado.sav', 'wb'))
pickle.dump(classificador_arvore, open('arvore_finalizada.sav', 'wb'))
pickle.dump(classificador_svm, open('svm_finalizado.sav', 'wb'))


## Carregar um Classificador já Treinado

In [ ]:
rede_neural = pickle.load(open('rede_neural_finalizado.sav', 'rb'))
arvore = pickle.load(open('arvore_finalizada.sav', 'rb'))
smv = pickle.load(open('svm_finalizado.sav', 'rb'))

In [ ]:
novo_registro = x_credit[0]
novo_registro

In [ ]:
novo_registro = novo_registro.reshape(1,-1)

In [ ]:
rede_neural.predict(novo_registro)

In [ ]:
arvore.predict(novo_registro)

In [ ]:
svm.predict(novo_registro)